In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [0]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# input data
batch_size =200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

Torch 1.1.0 CUDA 10.0.130
Device: cuda:0


In [0]:
dataset = pd.read_csv('/content/gdrive/My Drive/Colab/creditcard.csv')
X = dataset.iloc[:,1:-1]
Y = dataset.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)
train_target = torch.tensor(y_train.values.astype(np.float32))
train = torch.tensor(x_train.values.astype(np.float32))
train_tensor = torch.utils.data.TensorDataset(train, train_target) 
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
test_target = torch.tensor(y_test.values.astype(np.float32))
test = torch.tensor(x_test.values.astype(np.float32))
test_tensor = torch.utils.data.TensorDataset(test, test_target) 
test_loader = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)

In [0]:
x_test.shape

(85443, 29)

In [0]:
#Encoder
class Encoder_net(nn.Module):  
    def __init__(self):
        super(Encoder_net, self).__init__()
        self.l1 = nn.Linear(29, 20)
        self.l2 = nn.Linear(20, 10)
        self.l3 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = F.dropout(self.l1(x), p=0.25, training=self.training)
        x = F.elu(x)
        x = F.dropout(self.l2(x), p=0.25, training=self.training)
        x = F.elu(x)
        x = self.l3(x)
        return x
# Decoder
class Decoder_net(nn.Module):  
    def __init__(self):
        super(Decoder_net, self).__init__()
        self.l1 = nn.Linear(2, 10)
        self.l2 = nn.Linear(10, 20)
        self.l3 = nn.Linear(20, 29)
    def forward(self, x):
        x = F.elu(self.l1(x), 0.2)
        x = F.elu(self.l2(x), 0.2)
        x = F.elu(self.l3(x), 0.2)
        return torch.sigmoid(x)
# Discriminator
class Discriminant(nn.Module):  
    def __init__(self):
        super(Discriminant, self).__init__()
        self.l1 = nn.Linear(2, 10)
        self.l2 = nn.Linear(10, 10)
        self.l3 = nn.Linear(10, 2)
    def forward(self, x):
        x = F.dropout(self.l1(x), p=0.3, training=self.training)
        x = F.elu(x)
        x = F.dropout(self.l2(x), p=0.3, training=self.training)
        x = F.elu(self.l3(x))
        return torch.sigmoid(x)
      #F.softmax(x,dim=1)

In [0]:
def generate_(batch_size):
    return torch.from_numpy(np.random.multivariate_normal(mean=[2,4],cov =[[1.25,0],[0,1]],size=batch_size)).type(torch.float)

encoder = Encoder_net()
decoder = Decoder_net()
discriminant = Discriminant()
# learning parameter
lr1 = 1e-1
lr2 = 1e-3
#encode optimizers
optim_dec = torch.optim.Adam(decoder.parameters(), lr=lr1)
#decode optimizers
optim_enc = torch.optim.Adam(encoder.parameters(), lr=lr1)
# encode regularizing optimizers
optim_enc_gen = torch.optim.Adam(encoder.parameters(), lr=lr2)
# discriminant regularizing optimizers
optim_disc = torch.optim.Adam(discriminant.parameters(), lr=lr2)

In [0]:
# Start training
total_step = len(train_loader)
diss_loss =[]
enco_loss =[]
accuracy = []
num_epochs = 30

loss= nn.BCELoss()
loss1 = nn.MSELoss()
loss2 = nn.BCEWithLogitsLoss()
loss3 = nn.HingeEmbeddingLoss()
loss4 = nn.SmoothL1Loss()
loss5 = nn.SoftMarginLoss()
loss6 = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    diss =0
    enco =0
    train_correct =0
    for i ,(images,target) in enumerate(train_loader):
        decoder.train()
        batch_size = images.size(0)
        images = images.view(batch_size, -1)
        images = Variable(images,requires_grad =False)
        #labels
        r_labels = np.zeros(shape=(batch_size,2))
        r_labels[:,0] =1.0
        r_labels = torch.from_numpy(r_labels).type(torch.FloatTensor)
        r_labels = Variable(r_labels,requires_grad=False)

        f_labels = np.zeros(shape=(batch_size,2))
        f_labels[:,1] = 1.0
        f_labels = torch.from_numpy(f_labels).type(torch.FloatTensor)
        f_labels = Variable(f_labels,requires_grad=False)

        # set grad to zero
        optim_dec.zero_grad()
        optim_enc.zero_grad()
        optim_disc.zero_grad()
        optim_enc_gen.zero_grad()
    
    
        #reconstruction loss
        z_sample = encoder(images)   #encode to z
        X_sample = decoder(z_sample) #decode to X reconstruction
        recon_loss = loss4(X_sample,images) 
        train_correct1 = (r_labels == f_labels).sum()
        #accuracy = 100 * correct / 200
        train_correct += train_correct1

        # reconstruction loss train Encoder/Decoder
        recon_loss.backward()
        optim_enc.step()
        optim_dec.step()
        decoder.eval() 

        # Discriminator fake/real data
        F_data = Variable(generate_(batch_size))
        F_prediction = discriminant(F_data)  
        R_images = encoder(images)              
        R_prediction = discriminant(R_images)



        ## Discriminator loss train fake/real data
        Dloss = loss4(F_prediction,f_labels) + loss4(R_prediction,r_labels)
        Dloss.backward(retain_graph=True)
        optim_disc.step()

        ## Encoder/Generator
        Gloss = loss4(F_prediction,f_labels) + loss4(R_prediction,r_labels)
        Gloss.backward()
        optim_enc_gen.step()
        enco +=0.01*recon_loss.item()
        diss +=0.02*Dloss.item()
    
        # accuracy
        if (i+1) % 997 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Recon_loss is: {:.3f}, Discri_loss is: {:.3f}, accuracy: {}' 
                      .format(epoch+1, num_epochs, i+1, total_step, recon_loss.item(),Dloss.item(),train_correct1/200))
    enco_loss.append(enco)
    diss_loss.append(diss)
    accuracy.append(train_correct*100/len(train))

Epoch [1/30], Step [997/997], Recon_loss is: 4.088, Discri_loss is: 0.036, accuracy: 0
Epoch [2/30], Step [997/997], Recon_loss is: 3.215, Discri_loss is: 0.036, accuracy: 0
Epoch [3/30], Step [997/997], Recon_loss is: 3.030, Discri_loss is: 0.036, accuracy: 0
Epoch [4/30], Step [997/997], Recon_loss is: 2.839, Discri_loss is: 0.036, accuracy: 0
Epoch [5/30], Step [997/997], Recon_loss is: 3.814, Discri_loss is: 0.036, accuracy: 0
Epoch [6/30], Step [997/997], Recon_loss is: 3.184, Discri_loss is: 0.036, accuracy: 0
Epoch [7/30], Step [997/997], Recon_loss is: 3.119, Discri_loss is: 0.036, accuracy: 0
Epoch [8/30], Step [997/997], Recon_loss is: 3.445, Discri_loss is: 0.036, accuracy: 0
Epoch [9/30], Step [997/997], Recon_loss is: 3.077, Discri_loss is: 0.036, accuracy: 0
Epoch [10/30], Step [997/997], Recon_loss is: 3.644, Discri_loss is: 0.036, accuracy: 0
Epoch [11/30], Step [997/997], Recon_loss is: 4.274, Discri_loss is: 0.036, accuracy: 0
Epoch [12/30], Step [997/997], Recon_loss